<a href="https://colab.research.google.com/github/arvieira/bootcamp_projeto_final/blob/main/notebooks/CriacaoDoModelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importação das bibliotecas necessárias para o desenvolvimento dos modelos
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.dummy import DummyClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
import xgboost as xgb
from scipy.stats import uniform, randint
from joblib import dump
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# Importação dos dados limpos pelo notebook de Análise Exploratória
data = pd.read_csv('https://github.com/arvieira/bootcamp_projeto_final/blob/main/data/ready_data.csv?raw=true')
data = data.drop(['Unnamed: 0', 'PATIENT_VISIT_IDENTIFIER', 'WINDOW'], axis=1)
data.head()

,AGE_ABOVE65,AGE_PERCENTIL,GENDER,DISEASE GROUPING 1,DISEASE GROUPING 2,DISEASE GROUPING 3,DISEASE GROUPING 4,DISEASE GROUPING 5,DISEASE GROUPING 6,HTN,IMMUNOCOMPROMISED,OTHER,ALBUMIN_MEAN,BE_ARTERIAL_MEAN,BE_VENOUS_MEAN,BIC_ARTERIAL_MEAN,BIC_VENOUS_MEAN,BILLIRUBIN_MEAN,BLAST_MEAN,CALCIUM_MEAN,CREATININ_MEAN,FFA_MEAN,GGT_MEAN,GLUCOSE_MEAN,HEMATOCRITE_MEAN,INR_MEAN,LACTATE_MEAN,LEUKOCYTES_MEAN,LINFOCITOS_MEAN,NEUTROPHILES_MEAN,P02_ARTERIAL_MEAN,P02_VENOUS_MEAN,PC02_ARTERIAL_MEAN,PC02_VENOUS_MEAN,PCR_MEAN,PH_ARTERIAL_MEAN,PH_VENOUS_MEAN,PLATELETS_MEAN,POTASSIUM_MEAN,SAT02_ARTERIAL_MEAN,SAT02_VENOUS_MEAN,SODIUM_MEAN,TGO_MEAN,TGP_MEAN,TTPA_MEAN,UREA_MEAN,DIMER_MEAN,BLOODPRESSURE_DIASTOLIC_MEAN,BLOODPRESSURE_SISTOLIC_MEAN,HEART_RATE_MEAN,RESPIRATORY_RATE_MEAN,TEMPERATURE_MEAN,OXYGEN_SATURATION_MEAN,ICU
0,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,-1.000000,-1.000000,-0.317073,-0.317073,-0.938950,-1.0,0.183673,-0.868365,-0.742004,-0.945093,-0.891993,0.090147,-0.932246,1.000000,-0.835844,-0.914938,-0.868747,-0.170732,-0.704142,-0.779310,-0.754601,-0.875236,0.234043,0.363636,-0.540721,-0.518519,0.939394,0.345679,-0.028571,-0.997201,-0.990854,-0.825613,-0.836145,-0.994912,0.086420,-0.230769,-0.283019,-0.593220,-0.285714,0.736842,1
1,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,-1.000000,-1.000000,-0.317073,-0.317073,-0.938950,-1.0,0.183673,-0.868365,-0.742004,-0.945093,-0.891993,0.090147,-0.932246,1.000000,-0.835844,-0.914938,-0.868747,-0.170732,-0.704142,-0.779310,-0.754601,-0.875236,0.234043,0.363636,-0.540721,-0.518519,0.939394,0.345679,-0.028571,-0.997201,-0.990854,-0.825613,-0.836145,-0.994912,0.333333,-0.230769,-0.132075,-0.593220,0.535714,0.578947,1
2,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.605263,-1.000000,-1.000000,-0.317073,-0.317073,-0.938950,-1.0,0.183673,-0.868365,-0.742004,-0.945093,-0.891993,0.090147,-0.932246,1.000000,-0.835844,-0.914938,-0.868747,-0.170732,-0.704142,-0.779310,-0.754601,-0.875236,0.234043,0.363636,-0.540721,-0.518519,0.939394,0.345679,-0.028571,-0.997201,-0.990854,-0.825613,-0.836145,-0.994912,-0.243021,-0.338537,-0.213031,-0.317859,-0.107143,0.736842,1
3,1,5,0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.000000,-0.871658,-0.863874,-0.317073,-0.414634,-0.979069,-1.0,0.326531,-0.926398,-0.859275,-0.669393,-0.891993,-0.320755,-0.979925,-0.963023,-0.762843,-0.643154,-0.868747,-0.365854,-0.230769,-0.875862,-0.815951,-1.000000,0.574468,0.393939,-0.471295,-0.666667,0.848485,0.925926,0.142857,-0.999067,-0.983994,-0.846633,-0.836145,-0.996762,-0.243021,-0.338537,-0.213031,-0.317859,-0.107143,0.736842,1
4,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.605263,-1.000000,-1.000000,-0.317073,-0.317073,-0.938950,-1.0,0.357143,-0.912243,-0.742004,-0.958528,-0.780261,0.144654,-0.959849,1.000000,-0.382773,-0.908714,-0.412965,-0.170732,-0.704142,-0.779310,-0.754601,-0.939887,0.234043,0.363636,-0.399199,-0.703704,0.939394,0.345679,0.085714,-0.995428,-0.986662,-0.846633,-0.836145,-0.978029,-0.489712,-0.685470,-0.048218,-0.645951,0.357143,0.935673,1


In [ ]:
class ModelTester:
  def __init__(self, models, data, n=0, n_splits=0, n_repeats=0):
    # Modelos que serão executados
    self.models = models

    # Base de dados
    self.data = data

    # Como os dados já estão limpos e prontos para o treinamento, todas as colunas
    # exceto a que será prevista importa para o treino do modelo.
    self.x = data.drop(['ICU'], axis=1)

    # Coluna que será prevista. No caso, a idéia é tentar identificar se o paciente
    # que tenha uma das janelas analisadas, irá para a CTI ou não.
    self.y = data['ICU']

    # Parâmetros para execução do modelo ou cross validate
    # Se não passar o parâmetro, ele será considerado 0 e não será executada 
    # a etapa que precisa do parâmetro
    self.n = n
    self.n_splits = n_splits
    self.n_repeats = n_repeats

    # Lista para guardar os modelos e os resultados
    self.model_results = []

  
  # Getter do x 
  def get_x(self):
    return self.x

  # Getter do y
  def get_y(self):
    return self.y
  

  # Método estático para calcular de uma lista a média, o mínimo e o máximo do 
  # intervalo de confiança.
  @staticmethod
  def calculate_mean_confiance(values_list):
    # Calculando descritivos
    mean = np.mean(values_list)
    std = np.std(values_list)
    confiance_min = mean - (2 * std)
    confiance_max = mean + (2 * std)

    return mean, confiance_min, confiance_max


  # Método estático para imprimir resultados
  @staticmethod
  def print_results(metric, mean, confiance_min, confiance_max):
    # Imprimindo média
    print(f'{metric} médio: {mean}')

    # Imprimindo intervalo de confiança
    print(f'Intervalo de Confiança a 95%: {confiance_min} - {confiance_max}')


  # Embaralha o dataset e retorna o x e o y
  def do_shuffle_data(self):
    # Definindo um seed para a reprodutibilidade do experimento
    np.random.seed(234451)

    # Embaralhando os dados para poder realizar os folds para o cross-validate
    shuffle_data = self.data.sample(frac=1).reset_index(drop=True)
    x = shuffle_data.drop(['ICU'], axis=1)
    y = shuffle_data['ICU']

    return x, y


  # Método para executar o cross_validate
  def run_cross_validate(self, name, model):
    # Define o x e o y a partir do dataset
    x, y = self.do_shuffle_data()

    # Separando os diferentes cenários dos folds
    cv = RepeatedStratifiedKFold(n_splits=self.n_splits, n_repeats=self.n_repeats)

    # Fazendo a validação cruzada
    results = cross_validate(model, x, y, cv=cv, scoring='roc_auc')

    # Calculando as descrições do AUC
    self.auc_cross_mean, self.auc_cross_confiance_min, self.auc_cross_confiance_max = self.calculate_mean_confiance(results['test_score'])


  # Função para executar n vezes o modelo e calcular as métricas
  # AUC
  # Accuracy_score
  def run_n_models(self, name, model):
    # Essa será uma lista para guardar cada um dos AUCs de cada modelo criado.
    # Ao final extrairemos uma média com intervalo de confiança de 95% dos AUCs
    auc_list = []

    # Mesma coisa para o accuracy_score
    accuracy_score_list = []

    # Executando as n execuções do meu modelo
    for _ in range(self.n):
      # Divisão dos dados em treino e teste
      # Aqui utilizamos o parâmetro stratify para o y para manter os dados balanceados
      # quanto a pacientes que vão ou não para CTI nos dois grupos.
      x_train, x_test, y_train, y_test = train_test_split(self.x, self.y, stratify=self.y)

      # Treinando o modelo com os dados divididos
      model.fit(x_train, y_train)

      # Calculando o AUC
      prob_predict = model.predict_proba(x_test)
      auc = roc_auc_score(y_test, prob_predict[:, 1])

      # Adicionando o novo AUC à lista
      auc_list.append(auc)

      # Calculando accuracy_score
      y_prediction = model.predict(x_test)
      accuracy = accuracy_score(y_test, y_prediction)

      # Adicionando o novo accuracy_score à lista
      accuracy_score_list.append(accuracy)

    # Calculando as descrições do auc
    self.auc_mean, self.auc_confiance_min, self.auc_confiance_max = self.calculate_mean_confiance(auc_list)

    # Calculando as descrições do accuracy_score
    self.accuracy_mean, self.accuracy_confiance_min, self.accuracy_confiance_max = self.calculate_mean_confiance(accuracy_score_list)


  def report(self, name, model):
    # Header do relatório
    print('\n#######################################################################')
    print(f'Modelo: {name}\n')

    # Resultados de n execuções
    if self.n != 0:
      print(f'Executando {self.n} vezes o modelo:')
      self.print_results('AUC', self.auc_mean, self.auc_confiance_min, self.auc_confiance_max)
      self.print_results('Accuracy Score', self.accuracy_mean, self.accuracy_confiance_min, self.accuracy_confiance_max)
      print('')

    # Imprimindo resultados da validação cruzada
    if self.n_splits != 0 and self.n_repeats != 0:
      print(f'Executando Cross-Validate:')
      self.print_results('AUC', self.auc_cross_mean, self.auc_cross_confiance_min, self.auc_cross_confiance_max)

    # Footer do relatório
    print('#######################################################################\n')


  # Executa os testes para um dicionário de modelos
  def run(self):
    for model in self.models:
      if 'name' in model and 'model' in model:
        result = {
            'name': model['name'],
            'model': model['model'],            
        }

        if self.n != 0:
          self.run_n_models(model['name'], model['model'])
          result['auc_mean'] = self.auc_mean
          result['auc_confiance_min'] = self.auc_confiance_min
          result['auc_confiance_max'] = self.auc_confiance_max
          result['accuracy_mean'] = self.accuracy_mean
          result['accuracy_confiance_min'] = self.accuracy_confiance_min
          result['accuracy_confiance_max'] = self.accuracy_confiance_max
        else:
          result['auc_mean'] = 0
          result['auc_confiance_min'] = 0
          result['auc_confiance_max'] = 0
          result['accuracy_mean'] = 0
          result['accuracy_confiance_min'] = 0
          result['accuracy_confiance_max'] = 0

        if self.n_splits != 0 and self.n_repeats != 0:
          self.run_cross_validate(model['name'], model['model'])
          result['auc_cross_mean'] = self.auc_cross_mean
          result['auc_cross_confiance_min'] = self.auc_cross_confiance_min
          result['auc_cross_confiance_max'] = self.auc_cross_confiance_max
        else:
          result['auc_cross_mean'] = 0
          result['auc_cross_confiance_min'] = 0
          result['auc_cross_confiance_max'] = 0

        self.report(model['name'], model['model'])
        self.model_results.append(result)
      else:
        print('Erro: Faltando parâmetros na lista de modelos.')


  # Getter do melhor modelo encontrado
  def get_best_model(self):
    if self.n != 0 and self.n_splits != 0 and self.n_repeats != 0:
      best = {
          'name': None,
          'model': None,
          'auc_mean': 0,
          'auc_confiance_min': 0,
          'auc_confiance_max': 0,
          'accuracy_mean': 0,
          'accuracy_confiance_min': 0,
          'accuracy_confiance_max': 0,
          'auc_cross_mean': 0,
          'auc_cross_confiance_min': 0,
          'auc_cross_confiance_max': 0,
      }
      for model in self.model_results:
        if model['auc_mean'] > best['auc_mean'] and model['accuracy_mean'] > best['accuracy_mean'] and model['auc_cross_mean'] > best['auc_cross_mean']:
          best = model

      return best
    else:
      print('Erro: Para selecionar um melhor modelo, forneça os parâmetros n, n_split e n_repeats')


  # Otimização de hiperparâmetros pro Grid e Random
  def hiperparameters_search(self, name, model, params, n_iter=0):
    # Define o x e o y a partir do dataset
    x, y = self.do_shuffle_data()

    # Separando os folds pro corss validate
    cv = RepeatedStratifiedKFold(n_splits=self.n_splits, n_repeats=self.n_repeats)

    # Realizando a busca de parâmetros
    if n_iter == 0:
      print('Usando Grid Search para hiperparâmetros')
      search = GridSearchCV(model, param_grid=params, cv=cv, scoring='roc_auc', return_train_score=True)
    else:
      print('Usando Random Search para hiperparâmetros')
      search = RandomizedSearchCV(model, param_distributions=params, n_iter=n_iter, cv=cv, scoring='roc_auc', return_train_score=True)

    # Realizando o treino do modelo
    search.fit(x, y)

    # Criando o dataframe de resultados
    results = pd.DataFrame(search.cv_results_).iloc[search.best_index_]

    # Calculando descritivos de métricas
    auc_mean = results['mean_test_score']
    auc_train_mean = results['mean_train_score']
    auc_std = results['std_test_score']

    # Reportando o melhor score
    self.report_best_scores(search.cv_results_, 1)

    # Retornando o melhor modelo
    return search.best_estimator_


  # Método para reportar a melhor configuração de hiperparâmetros
  def report_best_scores(self, results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            auc_test_mean = results['mean_test_score'][candidate]
            std_test = results['std_test_score'][candidate]
            confiance_min = auc_test_mean - (2 * std_test)
            confiance_max = auc_test_mean + (2 * std_test)

            print("Modelo de número: {0}".format(i))
            print("AUC Médio: {0:.3f} (Desvio Padrão: {1:.3f})".format(auc_test_mean, std_test))
            print("Intervalo de Confiança a 95%: {0:.3f} - {1:.3f}".format(confiance_min, confiance_max))
            print("Parâmetros: {0}".format(results['params'][candidate]))
            print("")


In [ ]:
# Definindo dicionário para execução de modelos em sequência
models = [
          {
              'name': 'DummyClassifier',
              'model': DummyClassifier(),
          },
          {
              'name': 'LogisticRegression',
              'model': LogisticRegression(max_iter=10000)
          },
          {
              'name': 'DecisionTreeClassifier',
              'model': DecisionTreeClassifier()
          },
          {
              'name': 'XGBoostBinaryClassifier',
              'model': xgb.XGBClassifier(objective="binary:logistic", random_state=42)
          }
]

# Criando um objeto para testar os modelos
tester = ModelTester(models, data, n=10, n_splits=5, n_repeats=10)
tester.run()


#######################################################################
Modelo: DummyClassifier

Executando 10 vezes o modelo:
AUC médio: 0.5042345633905737
Intervalo de Confiança a 95%: 0.4546861195786198 - 0.5537830072025276
Accuracy Score médio: 0.561756373937677
Intervalo de Confiança a 95%: 0.5195058851274738 - 0.6040068627478802

Executando Cross-Validate:
AUC médio: 0.4967848970251716
Intervalo de Confiança a 95%: 0.4369776725045945 - 0.5565921215457487
#######################################################################


#######################################################################
Modelo: LogisticRegression

Executando 10 vezes o modelo:
AUC médio: 0.8583010595542564
Intervalo de Confiança a 95%: 0.8251361429099315 - 0.8914659761985813
Accuracy Score médio: 0.8048158640226628
Intervalo de Confiança a 95%: 0.7719302620715434 - 0.8377014659737823

Executando Cross-Validate:
AUC médio: 0.859570938215103
Intervalo de Confiança a 95%: 0.8188381119391707 - 0.900303764

In [ ]:
# Pegando o melhor modelo encontrado até o momento para realizar a customização
# dos hiperparâmetros
best_model = tester.get_best_model()
best_model['name']

'XGBoostBinaryClassifier'

In [ ]:
# Hiperparâmetros escolhidos como um grid para a busca do melhor modelo
grid_params = {
    "gamma": [0, 0.09, 1],
    "learning_rate": [0, 0.4, 1],
    "max_depth": [1, 6, 10], 
    "n_estimators": [10, 15, 20],
}

grid_best_model = tester.hiperparameters_search(best_model['name'], best_model['model'], grid_params)

Usando Grid Search para hiperparâmetros
Modelo de número: 1
AUC Médio: 0.976 (Desvio Padrão: 0.010)
Intervalo de Confiança a 95%: 0.957 - 0.995
Parâmetros: {'gamma': 0, 'learning_rate': 0.4, 'max_depth': 10, 'n_estimators': 20}



In [ ]:
# Hiperparâmetros escolhidos aleatoriamente
random_params = {
    "colsample_bytree": uniform(0.7, 0.3),
    "gamma": uniform(0, 0.5),
    "learning_rate": uniform(0.03, 0.3), # default 0.1 
    "max_depth": randint(2, 6), # default 3
    "n_estimators": randint(100, 150), # default 100
    "subsample": uniform(0.6, 0.4)
}

random_best_model = tester.hiperparameters_search(best_model['name'], best_model['model'], random_params, n_iter=3)

Usando Random Search para hiperparâmetros
Modelo de número: 1
AUC Médio: 0.980 (Desvio Padrão: 0.008)
Intervalo de Confiança a 95%: 0.964 - 0.997
Parâmetros: {'colsample_bytree': 0.8478292141990216, 'gamma': 0.20199896624003705, 'learning_rate': 0.295222614531939, 'max_depth': 4, 'n_estimators': 147, 'subsample': 0.6881861322455585}



In [ ]:
# Salvando o melhor modelo obtido pela busca em Grid
dump(grid_best_model, '/content/drive/MyDrive/Trabalho/CienciaDeDados/Bootcamp/Modulo05/ProjetoFinal/modelos/GridBestModel.joblib')

['/content/drive/MyDrive/Trabalho/CienciaDeDados/Bootcamp/Modulo05/ProjetoFinal/modelos/GridBestModel.joblib']

In [ ]:
# Salvando o melhor modelo obtido pela busca aleatória em intervalos
dump(random_best_model, '/content/drive/MyDrive/Trabalho/CienciaDeDados/Bootcamp/Modulo05/ProjetoFinal/modelos/RandomBestModel.joblib')

['/content/drive/MyDrive/Trabalho/CienciaDeDados/Bootcamp/Modulo05/ProjetoFinal/modelos/RandomBestModel.joblib']